<a href="https://colab.research.google.com/github/geraldoferreira1963/Classificacao/blob/main/Classificacao_Extrato_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn nltk gensim transformers -q
!pip install requests -q
!pip install scikeras -q
!pip install joblib -q  #


import joblib #
import requests  #
import tensorflow as tf  #
import time
import nltk
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import nltk
import string
from sklearn.naive_bayes import GaussianNB #
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import TextVectorization, Embedding, Conv1D, MaxPooling1D, Flatten, Dropout, Dense
from tensorflow.keras.utils import to_categorical #
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dropout, Dense
from tensorflow.keras.models import Model  #
from tensorflow.keras.layers import LSTM, Bidirectional #
from google.colab import files
from scikeras.wrappers import KerasClassifier  #
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Objetivo : Análise do Dataframe com o dataset carregado, e gerar um primeiro conjunto de estatísticas

In [2]:
def analisar_dataframe(df):
    # Contagem e Impressão do Conteúdo da Classificação
    classificacao_contagem = df['Classificacao'].value_counts()
    print("Distribuição da Classificacao:")

    # Contagem das classes de Classificacao
    data = [['Classificacao', 'Contagem']]
    for classificacao, contagem in classificacao_contagem.items():
        data.append([classificacao, contagem])

    # Formatação dos Dados
    col_width = max(len(str(word)) for row in data for word in row) + 2  # padding
    for row in data:
        print("|" + "|".join(str(word).ljust(col_width) for word in row) + "|")

    # Calcular e Imprimir estatísticas

    df['comprimento_conteudo'] = df['Operacao_Investimento'].apply(len)  #

    # Imprimir Estatísticas do tamanho do conteúdo
    print("\n\n Estatísticas do Comprimento do Conteúdo:")
    data = [
        ["Menor Comprimento", df['comprimento_conteudo'].min()],
        ["Maior Comprimento", df['comprimento_conteudo'].max()],
        ["Comprimento Mediano", df['comprimento_conteudo'].median()]
    ]

    # Formatar Tabela
    col_width = max(len(str(word)) for row in data for word in row) + 2  # padding
    for row in data:
        print("|" + "|".join(str(word).ljust(col_width) for word in row) + "|")

    comprimento_linha = df['comprimento_conteudo'].max()


    # Cálculo do Tamanho do Vocabulário
    all_words = ' '.join(df['Operacao_Investimento']).lower().split()
    unique_words = set(all_words)
    print("\n\n Número de Palavras:", len(unique_words))

    return(comprimento_linha,len(unique_words))

# Objetivo : Remoção de stop words, pontuações, opcionalmente lematização

In [3]:
def preprocess_text(text, lemmatize=True):

    tokens = nltk.word_tokenize(text.lower())

    # Remoção de pontuações
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remoção de Stop Words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lematização
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Objetivo : Gerar diversos embeddings, o método será passado como parametro (TF-IDF, Word2Vec, BERT)

In [4]:
def gerar_embedding(X_train, X_test, embedding_type='TF-IDF'):
    if embedding_type == 'TF-IDF':
        vectorizer = TfidfVectorizer(max_features=5000)
        X_train_emb = vectorizer.fit_transform(X_train)
        X_test_emb = vectorizer.transform(X_test)
    elif embedding_type == 'Word2Vec':
        sentences = [text.split() for text in X_train]
        model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
        X_train_emb = np.array([np.mean([model_w2v.wv[word] for word in text.split() if word in model_w2v.wv] or [np.zeros(100)], axis=0) for text in X_train])
        X_test_emb = np.array([np.mean([model_w2v.wv[word] for word in text.split() if word in text.split() if word in model_w2v.wv] or [np.zeros(100)], axis=0) for text in X_test])
    elif embedding_type == 'BERT':
        tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
        model_bert = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

        def get_bert_embedding(text):
            inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
            outputs = model_bert(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()
            # Reduce the number of features to 768 using PCA if necessary
            if embeddings.shape[1] > 768:
                from sklearn.decomposition import PCA
                pca = PCA(n_components=768)
                embeddings = pca.fit_transform(embeddings)
            return embeddings

        X_train_emb = np.array([get_bert_embedding(text) for text in X_train])
        X_test_emb = np.array([get_bert_embedding(text) for text in X_test])


    else:
        raise ValueError("Tipo de embedding não suportado.")

    return X_train_emb, X_test_emb

# Objetivo : Criar um modelo neural que utilizará como base uma camada CNN (número de filtros, e tamanho são parametros). Adicionalmente existem outros parametros (os quais serão Hiper Parametros) como : dropout e taxa de aprendizado

In [5]:
def criacao_modelo_cnn(num_filters=32, filter_size=3, learning_rate=0.01, dropout_rate=0.2):
    # Camada de Entrada ( entrada já estará vetorizada)
    input_layer = Input(shape=(max_length,))


    # Camada Embedding, pesos seráo aprendidos durante o treinamento
    embedding_layer = Embedding(input_dim=max_tokens, output_dim=128)(input_layer)

    # Camada Convolutional
    cnn_layer = Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu')(embedding_layer)

    # Camada Max pooling
    pooling_layer = MaxPooling1D(pool_size=2)(cnn_layer)

    # Flatten layer
    flatten_layer = Flatten()(pooling_layer)

    # Dropout layer
    dropout_layer = Dropout(dropout_rate)(flatten_layer)

    # Camada de Saída, Softmax, Classificação (Multi-Classe)
    output_layer = Dense(num_classes, activation='softmax')(dropout_layer)


    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])

    return model

# Objetivo : Criar um modelo neural que utilizará como base uma camada LSTM  (número de neurons, bidirecionalidade são parametros). Adicionalmente existem outros parametros (os quais serão Hiper Parametros) como : dropout e taxa de aprendizado

In [6]:
def criacao_modelo_lstm(num_neurons=64, bidirectional=True, learning_rate=0.01, dropout_rate=0.2):

    # Camada de Entrada ( entrada já estará vetorizada)
    input_layer = Input(shape=(max_length,))


    # Camada Embedding, pesos seráo aprendidos durante o treinamento
    embedding_layer = Embedding(input_dim=max_tokens, output_dim=128)(input_layer)

    # LSTM  (bidirectional ou unidirectional)
    if bidirectional:
        lstm_layer = Bidirectional(LSTM(num_neurons))(embedding_layer)
    else:
        lstm_layer = LSTM(num_neurons)(embedding_layer)

    # Dropout layer
    dropout_layer = Dropout(dropout_rate)(lstm_layer)

    # Camada de Saída com softmax ( multi-class classification )
    output_layer = Dense(num_classes, activation='softmax')(dropout_layer)


    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=['accuracy'])

    return model

# Objetivo : Imprimir Resultados dos Treinamentos

In [7]:
def imprimir_resultados_tabela(results):
    """Prints the results in a formatted table."""
    header = ["Modelo", "Embedding", "Acurácia"]
    col_widths = [max(len(str(x)) for x in col) for col in zip(*([header] + results))]  #

    # Cabeçario
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")
    print("|" + "|".join(str(x).center(width + 2) for x, width in zip(header, col_widths)) + "|")
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")

    # Linhas
    for row in results:
        print("|" + "|".join(str(x).center(width + 2) for x, width in zip(row, col_widths)) + "|")

    #
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")

In [8]:
def imprimir_resultados_tabela_II (results):
    """Prints the results in a formatted table."""
    header = ["Modelo", "Acuracia", "Acurácia"]
    col_widths = [max(len(str(x)) for x in col) for col in zip(*([header] + results))]  #

    # Cabeçario
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")
    print("|" + "|".join(str(x).center(width + 2) for x, width in zip(header, col_widths)) + "|")
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")

    # Linhas
    for row in results:
        print("|" + "|".join(str(x).center(width + 2) for x, width in zip(row, col_widths)) + "|")

    #
    print("+" + "+".join("-" * (width + 2) for width in col_widths) + "+")

In [9]:
def carregar_modelo_do_github(model_name, embedding_name):
    filename = f"{model_name}_{embedding_name}_best_model.pkl"
    github_url = f"https://raw.githubusercontent.com/geraldoferreira1963/Classificacao/main/{filename}"  #  GitHub URL

    # Verifica se o GitHub
    response = requests.get(github_url)
    if response.status_code == 200:
        # File existe no GitHub download e carrega
        with open(filename, "wb") as f:
            f.write(response.content)
        model = joblib.load(filename)
        print(f"Carregando modelo  {model_name} do GitHub.")
    else:
        # Arquivo não esta no GitHub, mensagem erro
        raise FileNotFoundError(f"Arquivo {filename} não encontrado no GitHub.")

    return model

# Carga do Dataset

In [10]:

url = 'https://raw.githubusercontent.com/geraldoferreira1963/Analise_Sentimento/main/carga_inicial_4.xlsx'

response = requests.get(url)

with open('carga_inicial_4.xlsx', 'wb') as file:
    file.write(response.content)

df = pd.read_excel('carga_inicial_4.xlsx', engine='openpyxl') #

df.head()

,Operacao_Investimento,Classificacao
0,Câmbio Padrão : R$5977.29,Operacao_Outras
1,Câmbio Padrão : R$10460.25,Operacao_Outras
2,Câmbio Padrão : R$10460.25,Operacao_Outras
3,Câmbio Padrão : R$9962.14,Operacao_Outras
4,Compra de 60 LTC a $ 43.3,Operacao_Bolsa


# Análise Inicial do Dataset

In [11]:
# Verificação Inicial do dataset
max_length, unique_words = analisar_dataframe(df)  #

Distribuição da Classificacao:
|Classificacao    |Contagem         |
|Imposto          |870              |
|Rendimento_Acao  |542              |
|Rendimento_FII   |462              |
|Operacao_Bolsa   |351              |
|Operacao_Outras  |314              |
|Aluguel          |245              |
|Taxa             |209              |
|Resgate_RF       |160              |
|Operacao_RF      |142              |
|Rendimento_RF    |48               |


 Estatísticas do Comprimento do Conteúdo:
|Menor Comprimento    |12                   |
|Maior Comprimento    |80                   |
|Comprimento Mediano  |41.0                 |


 Número de Palavras: 1291


# Modelos Não Neurais - Preparação dos Dados

In [12]:
# Preprocessamento da coluna Operação Investimento remoção de pontuação, stop words
df['Operacao_Investimento'] = df['Operacao_Investimento'].apply(preprocess_text)


# Divisão dos dados
X = df['Operacao_Investimento'].apply(preprocess_text)
y = df['Classificacao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Geração dos Embeddings
X_train_tfidf, X_test_tfidf = gerar_embedding(X_train, X_test, embedding_type='TF-IDF')
X_train_w2v, X_test_w2v = gerar_embedding(X_train, X_test, embedding_type='Word2Vec')
X_train_bert, X_test_bert = gerar_embedding(X_train, X_test, embedding_type='BERT')

# Modelos e Hiper Parametros
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC()
}

# Hiper Parametros
param_grids = {
    'Logistic Regression': {
        'solver': ['liblinear', 'saga'],
        'C': [0.1, 1, 10]
    },
    'SVM': {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10]
    }
}

embeddings = {
    'TF-IDF': (X_train_tfidf, X_test_tfidf),
    'Word2Vec': (X_train_w2v, X_test_w2v),
    'BERT': (X_train_bert.squeeze(1), X_test_bert.squeeze(1))
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

# Modelos Não Neurais - Treinamento

In [13]:
trained_models = {}
for model_name, model in models.items():
    for embedding_name, (X_train_emb, X_test_emb) in embeddings.items():
        start_time = time.time()  # Iniciando contagem do tempo

        if model_name in param_grids:
            grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy')
            grid_search.fit(X_train_emb, y_train)
            model = grid_search.best_estimator_
            print(f"Melhores Hiper Parametros: {grid_search.best_params_}")

            # Salvando o o melhor modelo e seus hiper parametros
            filename = f"{model_name}_{embedding_name}_best_model.pkl"
            joblib.dump(model, filename)
            files.download(filename)
            print(f"Melhor modelo {model_name} com  {embedding_name} salvo no  {filename}")


        if model_name == 'Naive Bayes' and embedding_name in ['TF-IDF']:
            X_train_emb = X_train_emb.toarray()
            X_test_emb = X_test_emb.toarray()

        #
        model.fit(X_train_emb, y_train)  # Treinamento

        end_time = time.time()  # Fim da contagem do tempo
        training_time = end_time - start_time  # Calcular tempo

        print(f"Tempo de Treinamento do modelo {model_name} ({embedding_name}): {training_time:.4f} segundos")

        # Armazenar o Modelo
        trained_models[(model_name, embedding_name)] = model

Melhores Hiper Parametros: {'C': 10, 'solver': 'saga'}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Melhor modelo Logistic Regression com  TF-IDF salvo no  Logistic Regression_TF-IDF_best_model.pkl
Tempo de Treinamento do modelo Logistic Regression (TF-IDF): 2.8035 segundos
Melhores Hiper Parametros: {'C': 10, 'solver': 'saga'}


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Melhor modelo SVM com  Word2Vec salvo no  SVM_Word2Vec_best_model.pkl
Tempo de Treinamento do modelo SVM (Word2Vec): 12.3439 segundos
Melhores Hiper Parametros: {'C': 1, 'kernel': 'linear'}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Melhor modelo SVM com  BERT salvo no  SVM_BERT_best_model.pkl
Tempo de Treinamento do modelo SVM (BERT): 17.6080 segundos


# Modelos Não Neurais - Avaliação e Impressão dos Resultados

In [14]:
results = []

for (model_name, embedding_name), model in trained_models.items():

    print(f"Usando o modelo treinado {model_name} com {embedding_name}.")

    X_test_emb = embeddings[embedding_name][1]
    y_pred = model.predict(X_test_emb)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia para o modelo de {model_name} ({embedding_name}): {accuracy:.4f}")
    print(classification_report(y_test, y_pred, zero_division=0))

    results.append([model_name, embedding_name, accuracy])

# Impressão dos Resultados
imprimir_resultados_tabela(results)

Usando o modelo treinado Logistic Regression com TF-IDF.
Acurácia para o modelo de Logistic Regression (TF-IDF): 0.9895
                 precision    recall  f1-score   support

        Aluguel       1.00      1.00      1.00        53
        Imposto       0.99      1.00      1.00       167
 Operacao_Bolsa       0.99      1.00      0.99        71
Operacao_Outras       1.00      0.98      0.99        61
    Operacao_RF       1.00      0.92      0.96        25
Rendimento_Acao       0.97      1.00      0.98       115
 Rendimento_FII       1.00      0.99      0.99        86
  Rendimento_RF       0.89      0.80      0.84        10
     Resgate_RF       1.00      0.97      0.98        31
           Taxa       1.00      1.00      1.00        50

       accuracy                           0.99       669
      macro avg       0.98      0.97      0.97       669
   weighted avg       0.99      0.99      0.99       669

Usando o modelo treinado Logistic Regression com Word2Vec.
Acurácia para o mode

# Modelos Não Neurais - Predições

In [20]:
model_files = [
    "Logistic Regression_TF-IDF_best_model.pkl",
    "SVM_BERT_best_model.pkl",
    "SVM_TF-IDF_best_model.pkl",
    "SVM_Word2Vec_best_model.pkl",
    "Logistic Regression_BERT_best_model.pkl",
    "Logistic Regression_Word2Vec_best_model.pkl"
]

for model_file in model_files:
    parts = model_file.split("_")
    model_name = parts[0]  # e.g., "Logistic Regression"
    if parts[1] != "best":
        embedding_name = parts[1]  # e.g., "TF-IDF"
    else:
        embedding_name = parts[0]

    try:
        # Carregar o modelo usando carregar_modelo_do_github:
        model = carregar_modelo_do_github(model_name, embedding_name)
    except FileNotFoundError:
        print(f"Error na carga do modelo {model_file} do GitHub.")
        continue

    X_test_emb = embeddings[embedding_name][1]

    # 4. Predição:
    y_pred = model.predict(X_test_emb)

    # 5. Avaliação:
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia do modelo {model_name} ({embedding_name}): {accuracy:.4f}")
    print(classification_report(y_test, y_pred, zero_division=0))

Carregando modelo  Logistic Regression do GitHub.
Acurácia do modelo Logistic Regression (TF-IDF): 0.9895
                 precision    recall  f1-score   support

        Aluguel       1.00      1.00      1.00        53
        Imposto       0.99      1.00      1.00       167
 Operacao_Bolsa       0.97      1.00      0.99        71
Operacao_Outras       1.00      0.98      0.99        61
    Operacao_RF       1.00      0.92      0.96        25
Rendimento_Acao       0.97      1.00      0.98       115
 Rendimento_FII       1.00      0.99      0.99        86
  Rendimento_RF       1.00      0.80      0.89        10
     Resgate_RF       1.00      0.97      0.98        31
           Taxa       1.00      1.00      1.00        50

       accuracy                           0.99       669
      macro avg       0.99      0.97      0.98       669
   weighted avg       0.99      0.99      0.99       669

Carregando modelo  SVM do GitHub.
Acurácia do modelo SVM (BERT): 0.9880
                 prec

# Modelos Neurais - Preparação dos Dados

In [16]:
df['Operacao_Investimento'] = df['Operacao_Investimento'].apply(preprocess_text)

#
all_classes = df['Classificacao'].unique()

# Gerando Encoders
label_encoder = LabelEncoder()
label_encoder.fit(all_classes)

# Divisão dos Dados
X = df['Operacao_Investimento']
y = df['Classificacao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vetorização da Operação Investimento
max_tokens = unique_words
max_length = max_length
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=max_length)
vectorizer.adapt(X_train)


# Transformando os dados de treinamento e teste em vetores
X_train_vec = vectorizer(X_train).numpy()
X_test_vec = vectorizer(X_test).numpy()

# Transformandos os dados de Label (Classificação ) em representações númericas
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Realizando o one-hot encoding de Classificação
y_train_encoded = to_categorical(y_train_encoded)
y_test_encoded = to_categorical(y_test_encoded)


num_classes = y_train_encoded.shape[1]

# Models e  Hiper Parametros

models = {
    'CNN': {
        'create_model': criacao_modelo_cnn,
        'param_grid': {
          #  'model__num_filters': [32, 64], # comentado decorrente da falta de memória para considerar este parametro no espaço de busca
         #   'model__filter_size': [3, 5],   # comentado decorrente da falta de memória para considerar este parametro no espaço de busca
            'model__learning_rate': [0.01, 0.001],
            'model__dropout_rate': [0.2, 0.5],
            'batch_size': [32, 64],
            'epochs': [10, 20]
        }
    },
    'LSTM': {
        'create_model': criacao_modelo_lstm,
        'param_grid': {
            'model__num_neurons': [32, 64],
           # 'model__bidirectional': [True, False],
            'model__learning_rate': [0.01, 0.001],
            'model__dropout_rate': [0.2, 0.5],
            'batch_size': [32, 64],
            'epochs': [10, 20]
        }
    }
}

# Modelos Neurais - Treinamento

In [17]:
best_models = {}

for model_name, model_info in models.items():
    print(f"Treinamento do modelo  {model_name} ...")
    best_accuracy = 0.0  # Inicializando

    # Iniciando a busca os melhores hiper parametros
    model = KerasClassifier(model=model_info['create_model'])
    grid_search = GridSearchCV(estimator=model, param_grid=model_info['param_grid'], cv=5, scoring='accuracy',verbose =0)
    grid_result = grid_search.fit(X_train_vec, y_train_encoded, verbose =0)

    # Imprimeir os melhores hiper parametros
    print(f"Melhores Hiper parametros para o  {model_name}: {grid_result.best_params_}")

    # Atualizar melhor acurácia e melhor modelo
    current_accuracy = grid_result.best_score_
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_models[model_name] = grid_result.best_estimator_  # Store the best KerasClassifier

# 2. Gerando os arquivos com os melhores modelos (CNN e LSTM)
for model_name, best_model in best_models.items():
    filename = f"{model_name}_best_model.keras"
    best_model.model_.save(filename)  # Salvando o modelo
    print(f"Melhor modelo {model_name} salvo em {filename}")
    files.download(filename)

Treinamento do modelo  CNN ...
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Melhores Hiper parametros para o  CNN: {'batch_size': 32, 'epochs': 20, 'model__dropout_rate': 0.5, 'model__learning_rate': 0.001}
Treinamento do modelo  LSTM ...
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/st

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Melhores Hiper parametros para o  LSTM: {'batch_size': 32, 'epochs': 20, 'model__dropout_rate': 0.2, 'model__learning_rate': 0.001, 'model__num_neurons': 64}
Melhor modelo CNN salvo em CNN_best_model.keras


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Melhor modelo LSTM salvo em LSTM_best_model.keras


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Modelos Neurais - Avaliação e Impressão dos Resultados

In [18]:
results = []  # Lista para armazenar os resultados

for model_name, best_model in best_models.items():
    filename = f"{model_name}_best_model.keras"

    # Carga dos melhores modelos (ou do arquivo, ou de best model)
    #trained_keras_model = tf.keras.models.load_model(filename)

    filename = f"{model_name}_best_model.keras"
    github_url = f"https://raw.githubusercontent.com/geraldoferreira1963/Classificacao/main/{filename}"  #  GitHub URL

    # Verifica se o arquivo esta no Github
    response = requests.get(github_url)
    if response.status_code == 200:
        # Arquivo esta no  GitHub, download e carrega
        with open(filename, "wb") as f:
            f.write(response.content)
        trained_keras_model = tf.keras.models.load_model(filename)
        print(f"Carregando modelo {model_name} do GitHub.")
    else:
        # Arquivo não esta no  GitHub, best_model
        trained_keras_model = best_model.model_  # Use o modelo do best model
        print(f"Usando melhor modelo {model_name} do tunning dos hiper parametros.")


    # Avaliar o modelo
    _, accuracy = trained_keras_model.evaluate(X_test_vec, y_test_encoded, verbose=1)
    print(f"Acurácia para o  {model_name}: {accuracy:.4f}")

    # Realizando a Predição
    y_pred_probs = trained_keras_model.predict(X_test_vec)  #
    y_pred = np.argmax(y_pred_probs, axis=1)      #
    y_pred = label_encoder.inverse_transform(y_pred) #

    # Gerar e Imprimir a Classificação do Report
    print(classification_report(y_test, y_pred))

    # Atualizando resultados
    results.append([model_name, accuracy])

#Impressão e Formatação dos Resultados
imprimir_resultados_tabela_II(results)

Usando melhor modelo CNN do tunning dos hiper parametros.
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.9850 - loss: 0.0574
Acurácia para o  CNN: 0.9836
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
                 precision    recall  f1-score   support

        Aluguel       1.00      1.00      1.00        53
        Imposto       0.98      1.00      0.99       167
 Operacao_Bolsa       0.97      1.00      0.99        71
Operacao_Outras       1.00      0.95      0.97        61
    Operacao_RF       1.00      0.92      0.96        25
Rendimento_Acao       0.97      0.99      0.98       115
 Rendimento_FII       0.98      0.99      0.98        86
  Rendimento_RF       1.00      0.80      0.89        10
     Resgate_RF       1.00      0.94      0.97        31
           Taxa       0.98      1.00      0.99        50

       accuracy                           0.98       669
      macro avg       0.99      0.96      0.97       669
   weighted avg       0.98      0.98      0.98       669

# Modelos Neurais - Predições

In [19]:
model_files = [
    "LSTM_best_model.keras",
    "CNN_best_model.keras"
]

for model_file in model_files:

    model_name = model_file.split("_")[0]  # e.g., "LSTM"

    # 2. Carga do modelo  GitHub:
    github_url = f"https://raw.githubusercontent.com/geraldoferreira1963/Classificacao/main/{model_file}"
    response = requests.get(github_url)

    if response.status_code == 200:
        with open(model_file, "wb") as f:
            f.write(response.content)

        if os.path.exists(model_file):
            model = tf.keras.models.load_model(model_file)
            print(f"Modelo  {model_name} carregado do GitHub.")
        else:
            print(f"Error: {model_file} não encontrado.")
            continue
    else:
        print(f"Erro na carga do {model_file} do GitHub.")
        continue

    # 3. Predição
    y_pred_probs = model.predict(X_test_vec)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_pred = label_encoder.inverse_transform(y_pred)

    # 4. Avaliação :
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia do modelo  {model_name}: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

Modelo  LSTM carregado do GitHub.
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Acurácia do modelo  LSTM: 0.9925
                 precision    recall  f1-score   support

        Aluguel       1.00      1.00      1.00        53
        Imposto       1.00      1.00      1.00       167
 Operacao_Bolsa       1.00      1.00      1.00        71
Operacao_Outras       1.00      0.97      0.98        61
    Operacao_RF       1.00      1.00      1.00        25
Rendimento_Acao       0.97      0.99      0.98       115
 Rendimento_FII       0.99      1.00      0.99        86
  Rendimento_RF       1.00      0.80      0.89        10
     Resgate_RF       1.00      1.00      1.00        31
           Taxa       0.98      1.00      0.99        50

       accuracy                           0.99       669
      macro avg       0.99      0.98      0.98       669
   weighted avg       0.99      0.99      0.99       669

Modelo  CNN carregado do GitHub.
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Acurácia do modelo  

Modelos Neurais - Predições